In [299]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, timedelta
import numpy as np

## Analisando dados de teste

In [312]:
df = pd.read_csv('idsc_test.csv')

In [313]:
# Pegando os pares numericos do snapshot_radar
rota = df['snapshot_radar'].head(1).values[0]
coordenadas = re.findall(r'-?\d+\.\d+', rota)
pares = [(float(coordenadas[i]), float(coordenadas[i+1])) for i in range(0, len(coordenadas), 2)]
map = pd.DataFrame(pares, columns=['Latitude', 'Longitude'])

In [314]:
df.head(1).T

,0
flightid,7945735584a3297121c4f5ae0de8ecd1
origem,SBKP
destino,SBSV
dt_dep,2023-05-30 11:36:03.000
hora_ref,2023-05-30 11:00:00.000
snapshot_radar,MULTIPOINT ((-0.8807200448127578 -0.5148453155...
path,http://satelite.cptec.inpe.br/repositoriogoes/...
hora_esperas,2023-05-30 10:00:00.000
esperas,0
aero_esperas,SBSV


In [316]:
colunas = ['dt_dep', 'hora_ref', 'hora_esperas', 'hora_metar', 'hora_metaf', 'hora_tcp']

# 1. Converter as colunas para datetime de uma só vez
df[colunas] = df[colunas].apply(pd.to_datetime)

# 2. Encontrar índices de linhas com 'hora_metaf' nula
null_indices = df.index[df['hora_metaf'].isna()]

# 3. Atualizar as colunas com base nas colunas 'hora_metar' e 'metar'
df.loc[null_indices, 'hora_metaf'] = df.loc[null_indices, 'hora_metar'] + timedelta(hours=1)
df.loc[null_indices, 'metaf'] = df.loc[null_indices, 'metar']
df.loc[null_indices, 'aero_metaf'] = df.loc[null_indices, 'aero_metar']

# 4. Converter as colunas para valores inteiros UNIX timestamp
# df[colunas] = df[colunas].apply(lambda x: x.dt.timestamp()).astype(int)

## Analisando dados da API

In [315]:
def read_and_process_csv(file_path, date_columns=None, rename_columns=None):
    df = pd.read_csv(file_path)
    print(df.info())
    
    if date_columns:
        for col in date_columns:
            df[col] = pd.to_datetime(df[col].apply(convert), format='%d/%m/%Y %H:%M:%S')
    if rename_columns:
        df = df.rename(columns=rename_columns)
    
    return df

# Define a função de conversão
def convert(timestamp):
    data_hora = datetime.fromtimestamp(float(timestamp)/1000.0)
    return data_hora.strftime('%d/%m/%Y %H:%M:%S')

### Bimtra

In [303]:
bimtra = read_and_process_csv('Dados/bimtra.csv', date_columns=['dt_dep', 'dt_arr'], rename_columns={'hora': 'hora_ref'})
bimtra['hora_ref'] = bimtra['dt_dep'].dt.floor('H')
bimtra

,flightid,origem,destino,dt_dep,dt_arr,hora_ref
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-05-31 21:44:57,2022-05-31 22:26:00,2022-05-31 21:00:00
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-05-31 21:57:01,2022-05-31 22:32:53,2022-05-31 21:00:00
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-05-31 21:26:52,2022-05-31 22:43:45,2022-05-31 21:00:00
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-05-31 21:58:40,2022-05-31 22:41:41,2022-05-31 21:00:00
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-05-31 22:00:38,2022-05-31 23:33:05,2022-05-31 22:00:00
...,...,...,...,...,...,...
300342,c4321a6a09b76969df53965fb5390f12,SBBR,SBRF,2023-05-13 09:14:19,2023-05-13 11:20:23,2023-05-13 09:00:00
300343,e4f5686cce45083566945c8946e5d120,SBBR,SBGR,2023-05-13 06:33:15,2023-05-13 07:58:41,2023-05-13 06:00:00
300344,04f65a7c44cf176215520b4c9b4eb37f,SBCF,SBRF,2023-05-13 13:53:53,2023-05-13 16:00:24,2023-05-13 13:00:00
300345,f72a0718e581d01c695faeeff35d81f0,SBSV,SBGR,2023-05-13 02:28:58,2023-05-13 04:54:41,2023-05-13 02:00:00


### Esperas

In [272]:
esperas = read_and_process_csv('Dados/esperas.csv', date_columns=['hora'], rename_columns={'hora': 'hora_esperas', 'aero': 'aero_esperas'})
esperas

,esperas,hora_esperas,aero_esperas
0,0,2022-05-31 21:00:00,SBBR
1,0,2022-05-31 22:00:00,SBBR
2,0,2022-05-31 23:00:00,SBBR
3,0,2022-06-01 00:00:00,SBBR
4,0,2022-06-01 01:00:00,SBBR
...,...,...,...
99931,0,2023-05-13 16:00:00,SBSV
99932,0,2023-05-13 17:00:00,SBSV
99933,0,2023-05-13 18:00:00,SBSV
99934,0,2023-05-13 19:00:00,SBSV


### METAF

In [274]:
metaf = read_and_process_csv('Dados/metaf.csv', date_columns=['hora'], rename_columns={'hora': 'hora_metaf', 'aero': 'aero_metaf'})
metaf

,hora_metaf,metaf,aero_metaf
0,2022-08-04 21:00:00,METAF SBBR 050000Z 05006KT CAVOK 18/04 Q1014=\n,SBBR
1,2022-08-04 22:00:00,METAF SBBR 050100Z 01006KT CAVOK 17/05 Q1016=\n,SBBR
2,2022-08-04 23:00:00,METAF SBBR 050200Z 35008KT CAVOK 17/05 Q1014=\n,SBBR
3,2022-08-05 00:00:00,METAF SBBR 050300Z 34007KT CAVOK 16/05 Q1015=\n,SBBR
4,2022-08-05 01:00:00,METAF SBBR 050400Z 34008KT CAVOK 16/05 Q1015=\n,SBBR
...,...,...,...
43933,2023-05-13 16:00:00,METAF SBSP 131900Z 14010KT 9999 OVC024 ...,SBSP
43934,2023-05-13 17:00:00,METAF SBSP 132000Z 14009KT 9999 BKN033 ...,SBSP
43935,2023-05-13 18:00:00,METAF SBSP 132100Z 14008KT 8000 OVC033 ...,SBSP
43936,2023-05-13 19:00:00,METAF SBSP 132200Z 14007KT 7000 OVC033 ...,SBSP


### METAR

In [275]:
metar = read_and_process_csv('Dados/metar.csv', date_columns=['hora'], rename_columns={'hora': 'hora_metar', 'aero': 'aero_metar'})
metar

,hora_metar,metar,aero_metar
0,2022-05-31 21:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,2022-05-31 22:00:00,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,2022-05-31 23:00:00,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,2022-06-01 00:00:00,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,2022-06-01 01:00:00,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
106739,2023-05-13 16:00:00,METAR SBSV 131900Z 16009KT 9999 BKN020 FEW021T...,SBSV
106740,2023-05-13 17:00:00,METAR SBSV 132000Z 13007KT 9999 BKN020 29/24 Q...,SBSV
106741,2023-05-13 18:00:00,METAR SBSV 132100Z 12005KT 9999 SCT020 FEW025T...,SBSV
106742,2023-05-13 19:00:00,METAR SBSV 132200Z 11006KT 9999 SCT020 FEW025T...,SBSV


### Satalite

In [285]:
satelite=pd.read_csv('Dados/satelite.csv')
satelite

,data,path,tamanho
0,2022-06-01 01:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1879673
1,2022-06-01 02:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1877693
2,2022-06-01 03:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1887299
3,2022-06-01 04:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1890950
4,2022-06-01 05:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1892776
...,...,...,...
8282,2023-05-13 19:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1830030
8283,2023-05-13 20:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1828890
8284,2023-05-13 21:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1823160
8285,2023-05-13 22:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1809551


### TCP

In [286]:
tcp = read_and_process_csv('Dados/tc-prev.csv', date_columns=['hora'], rename_columns={'hora': 'hora_tcp', 'aero': 'aero_tcp'})
tcp

,hora_tcp,troca,aero_tcp
0,2022-05-31 21:00:00,0,BR
1,2022-05-31 22:00:00,0,BR
2,2022-05-31 23:00:00,0,BR
3,2022-06-01 00:00:00,0,BR
4,2022-06-01 01:00:00,0,BR
...,...,...,...
99931,2023-05-13 16:00:00,0,SV
99932,2023-05-13 17:00:00,0,SV
99933,2023-05-13 18:00:00,0,SV
99934,2023-05-13 19:00:00,0,SV


### TCR

In [287]:
tcr = read_and_process_csv('Dados/tc-real.csv', date_columns=['hora'], rename_columns={'hora': 'hora_tcr', 'aero': 'aero_tcr'})
tcr

,hora_tcr,nova_cabeceira,antiga_cabeceira,aero_tcr
0,2022-06-01 11:14:03,32,03,FL
1,2022-06-01 15:51:10,18,12,RF
2,2022-06-01 16:12:25,16,13,CF
3,2022-06-01 16:12:38,15,26,GL
4,2022-06-01 16:12:57,15,26,GL
...,...,...,...,...
33426,2023-05-13 12:32:28,14,08,FL
33427,2023-05-13 19:06:36,17R,12,SP
33428,2023-05-13 12:41:44,15,26,CT
33429,2023-05-13 19:41:08,17R,12,SP


## Junção

Considerando a referência da coluna (`hora_ref`):

- `hora_espera` é ajustada para -1 hora.
- `hora_metaf` é ajustada para +1 hora.
- `hora_metar` permanece inalterada.
- `hora_tcp` é ajustada para +1 hora.
- `hora_tcr` permanece inalterada.

Em relação às colunas ausentes dos aeroportos:

- Em `aero_tcp` é o valor do destino.
- Em `aero_tcr` é o valor do destino.

In [ ]:
merge1 = pd.merge(bimtra, esperas, left_on=['hora_ref', 'destino'], right_on=['hora_esperas', 'aero_esperas'], how='inner')
merge2 = pd.merge(merge1, metaf, left_on=['hora_esperas', 'aero_esperas'], right_on=['hora_metaf', 'aero_metaf'], how='inner')
merge3 = pd.merge(merge2, metar, left_on=['hora_metaf', 'aero_metaf'], right_on=['hora_metaf', 'aero_metaf'], how='inner')

### Exemplo de como usar Metar

In [237]:
from metar import Metar
obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
print (obs.string())

station: KEWR
type: routine report, cycle 19 (automatic report)
time: Mon Sep 11 18:51:00 2023
temperature: 22.7 C
dew point: 21.5 C
wind: variable at 3 knots, gusting to 19 knots
peak wind: WNW at 28 knots at 18:17
wind shift: 18:12
visibility: 2 miles
visual range: on runway 04R, from 3000 to greater than 6000 feet
pressure: 1011.5 mb
weather: thunderstorm with rain; mist
sky: a few clouds at 1500 feet
     broken cumulonimbus at 4000 feet
     broken clouds at 6500 feet
     overcast at 20000 feet
sea-level pressure: 1011.4 mb
1-hour precipitation: 0.13in
remarks:
- Automated station (type 2)
- peak wind 28kt from 290 degrees at 18:17
- wind shift at 18:12
- frequent lightning (intracloud,cloud-to-cloud,cloud-to-ground)
- thunderstorm overhead and NW
- TSB05RAB22 -N-E MOV NE
METAR: METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T